<a href="https://colab.research.google.com/github/zakaria-narjis/Sms-text-classification/blob/main/sms_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-10-22 10:45:00--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2022-10-22 10:45:00 (11.1 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-10-22 10:45:00--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
train_dataset=tf.data.TextLineDataset(train_file_path)
valid_dataset=tf.data.TextLineDataset(test_file_path)

In [4]:
def process_function(dataset):
    ds=dataset.map(lambda x:tf.strings.split(x, sep='\t'))
    ds=ds.map(lambda x:(x[1],x[0]))
    ds=ds.map(lambda x,y:(x,tf.cast(tf.math.equal(y,'spam'),tf.int32)))
    return ds

In [5]:
train_dataset=process_function(train_dataset)
valid_dataset=process_function(valid_dataset)

In [6]:
for feature,label in train_dataset.take(1):
  print(feature)
  print(label)

tf.Tensor(b'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.', shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int32)


In [7]:
import tensorflow_hub as hub
model=keras.Sequential()
input=hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",trainable=True, dtype=tf.string, input_shape=[])
model.add(input)
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [8]:
early_stopping=keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)

In [9]:
batch_size=32
train_set=train_dataset.shuffle(10000).batch(batch_size).prefetch(1)
valid_set=valid_dataset.batch(batch_size).prefetch(1)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 256,863,617
Trainable params: 256,863,617
Non-trainable params: 0
_________________________________________________________________


In [11]:
for feature,label in train_set.take(1):
  print(feature)
  print(label)

tf.Tensor(
[b'thanks  and ! or bomb and date as my phone wanted to say!'
 b'hi chachi tried calling u now unable to reach u .. pl give me a missed cal once u c tiz msg  kanagu'
 b"well at this right i'm gonna have to get up and check today's steam sales/pee so text me when you want me to come get you"
 b'so gd got free ice cream... i oso wan...'
 b'haha yeah, 2 oz is kind of a shitload'
 b"why don't you go tell your friend you're not sure you want to live with him because he smokes too much then spend hours begging him to come smoke"
 b'dear good morning now only i am up' b"sorry i'm not free..."
 b"do you ever notice that when you're driving, anyone going slower than you is an idiot and everyone driving faster than you is a maniac?"
 b"honestly i've just made a lovely cup of tea and promptly dropped my keys in it and then burnt my fingers getting them out!"
 b'as usual..iam fine, happy &amp; doing well..:)'
 b'where do you need to go to get it?'
 b'say this slowly.? god,i love you &am

In [12]:
model.fit(train_set,validation_data=valid_set,epochs=1000,callbacks=[early_stopping])

Epoch 1/1000
131/131 [==============================] - 32s 175ms/step - loss: 0.1333 - accuracy: 0.9586 - val_loss: 0.0467 - val_accuracy: 0.9856
Epoch 2/1000
131/131 [==============================] - 21s 156ms/step - loss: 0.0223 - accuracy: 0.9928 - val_loss: 0.0560 - val_accuracy: 0.9777
Epoch 3/1000
131/131 [==============================] - 22s 163ms/step - loss: 0.0066 - accuracy: 0.9988 - val_loss: 0.0424 - val_accuracy: 0.9878
Epoch 4/1000
131/131 [==============================] - 21s 157ms/step - loss: 0.0017 - accuracy: 0.9998 - val_loss: 0.0595 - val_accuracy: 0.9871
Epoch 5/1000
131/131 [==============================] - 21s 157ms/step - loss: 2.8628e-04 - accuracy: 1.0000 - val_loss: 0.0528 - val_accuracy: 0.9864
Epoch 6/1000
131/131 [==============================] - 21s 156ms/step - loss: 1.4250e-04 - accuracy: 1.0000 - val_loss: 0.0541 - val_accuracy: 0.9878
Epoch 7/1000
131/131 [==============================] - 21s 156ms/step - loss: 9.7993e-05 - accuracy: 1.0000 -

In [13]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  proba=model(tf.constant([pred_text]),training=False) 
  proba=proba[0][0].numpy()
  prediction=[proba,['ham','spam'][float(proba)>0.5]]
  return prediction

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.0004456139, 'ham']


In [14]:
predict_message('you won 500$ call to claim your prize.')

[0.99611, 'spam']

In [15]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
